
### 🔑 Part 4 연습문제 정답

#### 연습문제 1 정답

In [ ]:
# 1. 새로운 기본 모델 추가
from sklearn.ensemble import GradientBoostingRegressor

base_estimators_new = [
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgboost', XGBRegressor(random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=50, random_state=42)) # Gradient Boosting 추가
]

# 2. 새로운 메타 모델 정의
meta_estimator_new = Lasso(alpha=1.0, random_state=42)

# 각 기본 모델을 위한 파이프라인 재생성
rf_pipeline_new = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', base_estimators_new[0][1])])
xgb_pipeline_new = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', base_estimators_new[1][1])])
gb_pipeline_new = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', base_estimators_new[2][1])])

stacking_estimators_new = [
    ('random_forest', rf_pipeline_new),
    ('xgboost', xgb_pipeline_new),
    ('gb', gb_pipeline_new)
]

# 3. 새로운 StackingRegressor 생성 및 학습
# 메타 모델이 선형 모델이므로 스케일링이 필요. 기존 preprocessor를 재사용
meta_pipeline_new = Pipeline(steps=[('scaler', StandardScaler()), ('lasso', meta_estimator_new)])

stacking_regressor_new = StackingRegressor(
    estimators=stacking_estimators_new,
    final_estimator=meta_pipeline_new,
    cv=5,
    n_jobs=-1
)

stacking_regressor_new.fit(X_train, y_train)

# 4. 성능 평가
y_pred_stack_new = stacking_regressor_new.predict(X_test)
rmse_stack_new = np.sqrt(mean_squared_error(y_test, y_pred_stack_new))
print(f"새로운 스태킹 앙상블(메타모델: Lasso)의 RMSE: {rmse_stack_new:.4f}")

# 이전 결과와 비교
print(f"기존 스태킹 앙상블(메타모델: Ridge)의 RMSE: {rmse_stack:.4f}")

#### 연습문제 2 정답

In [ ]:
# 1. test_size=0.5로 데이터 분할
X_train_a_half, X_valid_b_half, y_train_a_half, y_valid_b_half = train_test_split(
    X_train, y_train, test_size=0.5, random_state=42
)

print(f"기본 모델 훈련 데이터 크기 (50%): {X_train_a_half.shape[0]}")
print(f"메타 모델 훈련 데이터 크기 (50%): {X_valid_b_half.shape[0]}")

# 2. 동일한 기본 모델들을 train_a_half로 학습
trained_base_models_half = {}
for name, model in base_models:
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', model)])
    pipeline.fit(X_train_a_half, y_train_a_half)
    trained_base_models_half[name] = pipeline

# 3. 메타 특성 생성 및 메타 모델 학습
meta_features_valid_half = pd.DataFrame()
for name, model in trained_base_models_half.items():
    meta_features_valid_half[name] = model.predict(X_valid_b_half)

meta_model_blend_half = Ridge(random_state=42)
meta_model_blend_half.fit(meta_features_valid_half, y_valid_b_half)

# 4. 최종 예측 및 성능 평가
meta_features_test_half = pd.DataFrame()
for name, model in trained_base_models_half.items():
    meta_features_test_half[name] = model.predict(X_test)

y_pred_blend_half = meta_model_blend_half.predict(meta_features_test_half)
rmse_blend_half = np.sqrt(mean_squared_error(y_test, y_pred_blend_half))

print(f"\n블렌딩 앙상블(test_size=0.5)의 RMSE: {rmse_blend_half:.4f}")
print(f"기존 블렌딩 앙상블(test_size=0.3)의 RMSE: {rmse_blend:.4f}")

# 결과 분석:
# test_size를 0.3에서 0.5로 늘리면 기본 모델이 학습할 데이터(train_a)는 줄어들고,
# 메타 모델이 학습할 데이터(valid_b)는 늘어납니다.
# 기본 모델의 성능은 약간 저하될 수 있지만, 메타 모델은 더 많은 데이터를 기반으로 학습하므로
# 더 안정적인 조합 가중치를 찾을 수 있습니다. 최종 성능은 이 두 효과의 상충 관계에 따라 결정됩니다.
# 데이터셋이 작을수록 train_a의 크기를 줄이는 것이 성능에 더 큰 악영향을 미칠 수 있습니다.